Syntax natürlicher Sprachen, WS 2023/24

# 03 - Aufgabenblatt (Lösung)

In [1]:
import nltk
from nltk.tree import Tree
from nltk import CFG, Production, Nonterminal
import copy

## Aufgabe 1 - Phrasenstrukturgrammatik in X-Bar-Struktur

#### Wandeln Sie die gegebene minimale CFG mit flachen Regeln gemäß den folgenden Anweisungen schrittweise in X-Bar-Struktur um. Kopieren Sie jeweils die Grammatik von der vorherigen Teilaufgabe.


In [2]:
sent = 'der Hund jagt den langsamen Briefträger'

In [3]:
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    VP  -> V NP
    NP  -> DET N
    NP  -> DET ADJ N

    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger"
    ADJ -> "langsamen"
    V   -> "jagt"
""")

parser = nltk.ChartParser(grammar)
for tree in parser.parse(sent.split()):
    tree.pretty_print(unicodelines=True)

              S                                
     ┌────────┴────────┐                        
     │                 VP                      
     │        ┌────────┴──────┐                 
     NP       │               NP               
 ┌───┴───┐    │    ┌──────────┼──────────┐      
DET      N    V   DET        ADJ         N     
 │       │    │    │          │          │      
der     Hund jagt den     langsamen Briefträger



### a) NP-Adjunkt (ADJ) und NP-Spezifizierer (DET)

#### Wandeln Sie die NP-Regeln in X-Bar-Struktur um, um rekursive Erweiterung um Adjektive zu ermöglichen.


In [4]:
sent = 'der Hund jagt den langsamen schreienden Briefträger'

In [5]:
grammar = nltk.CFG.fromstring("""
    S  -> NP VP
    VP -> V NP
    
#NP-SPEZIFIZIERER:    
    NP  -> DET NOM
#NP-ADJUNKT:
    NOM -> ADJ NOM
#NP-KOMPLEMENT-REGEL (ohne Komplement, nur Kopf):
    NOM -> N
 
    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger"
    ADJ -> "langsamen" | "schreienden"
    V   -> "jagt"
""")

parser = nltk.ChartParser(grammar)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

                   S                                           
     ┌─────────────┴──────┐                                     
     │                    VP                                   
     │        ┌───────────┴──────┐                              
     │        │                  NP                            
     │        │    ┌─────────────┴───────┐                      
     │        │    │                    NOM                    
     │        │    │      ┌──────────────┴───────┐              
     NP       │    │      │                     NOM            
 ┌───┴───┐    │    │      │              ┌───────┴───────┐      
 │      NOM   │    │      │              │              NOM    
 │       │    │    │      │              │               │      
DET      N    V   DET    ADJ            ADJ              N     
 │       │    │    │      │              │               │      
der     Hund jagt den langsamen     schreienden     Briefträger



#### Gleichzeitig sollen (durch Verwendung einer nominalen X-Bar-Zwischenebene NOM) Überproduktionen folgender Art vermieden werden:

In [6]:
neg_sent = 'der Hund jagt den langsamen schreienden den Briefträger'

In [7]:
parser = nltk.ChartParser(grammar)
trees = list(parser.parse(neg_sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {neg_sent}")

no parse found for: der Hund jagt den langsamen schreienden den Briefträger


### b) VP-Komplement (Objekt-NP) und NP-Komplement (Genitiv-NP)

#### Wandeln Sie die VP-Regeln in X-Bar-Struktur um. 

#### Ergänzen Sie außerdem eine NP-Regel in X-Bar-Struktur für eine Genitiv-NP als Komplement der NP.

In [8]:
sent = 'der Hund jagt den Briefträger der Stadt der Briefträger'

In [9]:
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    
#VP-SPEZIFIZIERER (ohne Spez.):   
    VP  -> VERBAL 
#VP-KOMPLEMENT (transitives Verb):    
    VERBAL -> V NP
        
    NP  -> DET NOM
    NOM -> ADJ NOM
    NOM -> N
#neue NP-KOMPLEMENT-Regel (Genititv-NP):    
    NOM -> N NP
 
    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger" | "Stadt"
    ADJ -> "langsamen" | "schreienden"
    V   -> "jagt"
""")

parser = nltk.ChartParser(grammar)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

         S                                                            
     ┌───┴──────────┐                                                  
     │              VP                                                
     │              │                                                  
     │            VERBAL                                              
     │        ┌─────┴─────────────────┐                                
     │        │                       NP                              
     │        │     ┌─────────────────┴────┐                           
     │        │     │                     NOM                         
     │        │     │         ┌────────────┴────┐                      
     │        │     │         │                 NP                    
     │        │     │         │       ┌─────────┴───┐                  
     │        │     │         │       │            NOM                
     │        │     │         │       │    ┌────────┴───┐              

### c) mehr VP-Komplemente (Objekt und indirektes Objekt)

#### Ergänzen Sie eine VP-Komplement-Regel für das (fakultativ) ditransitive Verb *jemandem etwas übergeben*.

##### Beachten Sie, dass die hier auftretende Ambiguität (*der Stadt* als Dativ-VP-Komplement bzw. als Genitiv-NP-Komplement) auch in Ihrem Parsingresultat sichtbar wird (2 Ableitungsbäume).

In [10]:
sent = 'der Briefträger übergibt den Hund der Stadt'

In [11]:
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    
    VP  -> VERBAL 
    VERBAL -> V NP
#neue VP-KOMPLEMENT-Regel (ditransitives Verb = 2 Komplemente):        
    VERBAL -> V NP NP


    NP  -> DET NOM
    NOM -> ADJ NOM
    NOM -> N
    NOM -> N NP
 
    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger" | "Stadt"
    ADJ -> "langsamen" | "schreienden"
    V   -> "jagt" | "übergibt"
""")

parser = nltk.ChartParser(grammar)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

                              S                          
     ┌────────────────────────┴────────┐                  
     │                                 VP                
     │                                 │                  
     │                               VERBAL              
     │                 ┌──────────┬────┴─────────┐        
     NP                │          NP             NP      
 ┌───┴───────┐         │      ┌───┴────┐     ┌───┴────┐   
 │          NOM        │      │       NOM    │       NOM 
 │           │         │      │        │     │        │   
DET          N         V     DET       N    DET       N  
 │           │         │      │        │     │        │   
der     Briefträger übergibt den      Hund  der     Stadt

                              S                          
     ┌────────────────────────┴────┐                      
     │                             VP                    
     │                             │                      
     

### d) VP-Spezifizierer (AUX) mit Zusatzregel für invertiertes Komplement

#### Ergänzen Sie Regeln für ein Auxiliar als Spezifizierer der VP sowie für die dann notwendige invertierte Wortstellung von Objekt-NP und Vollverb.


In [12]:
sent = 'der Briefträger hat den Hund übergeben'

In [13]:
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    
    VP  -> VERBAL 
    VERBAL -> V NP
    VERBAL -> V NP NP
#neuer VP-SPEZIFIZIERER (Auxiliar):        
    VP -> AUX VERBAL
#neue VP-KOMPLEMENT-REGEL mit invertierter Wortstellung: 
    VERBAL -> NP V

    NP  -> DET NOM
    NOM -> ADJ NOM
    NOM -> N
    NOM -> N NP
 
    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger" | "Stadt"
    ADJ -> "langsamen" | "schreienden" | "ganzen"
    V   -> "jagt" | "übergibt"
#neue lexikalische Regeln für AUX:
    AUX -> "hat"
    V -> "übergeben"
""")

parser = nltk.ChartParser(grammar)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

                     S                          
     ┌───────────────┴───────┐                   
     │                       VP                 
     │               ┌───────┴────┐              
     │               │          VERBAL          
     │               │       ┌────┴────────┐     
     NP              │       NP            │    
 ┌───┴───────┐       │   ┌───┴────┐        │     
 │          NOM      │   │       NOM       │    
 │           │       │   │        │        │     
DET          N      AUX DET       N        V    
 │           │       │   │        │        │     
der     Briefträger hat den      Hund  übergeben



## Aufgabe 2 - Transformation in X-Bar-Struktur

### a) Transformieren Sie die Grammatik von Aufgabenblatt 02 im nominale Bereich in X-Bar-Struktur und testen Sie anschließend mit den Positiv- und Negativ-Sätzen. Geben Sie dabei jeweils den X-Bar-Regeltyp an.

#### HINWEIS: Alle PP-Attribute (inkl. *von*-PPs, die man - analog zu den  Genitiv-NPs - auch als Komplement auffassen kann) sollen hier vereinfacht als rekursive Adjunkte modelliert werden.

In [14]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    VP -> V NP NP
    PP -> P NP
    NP -> PROPN | PROPN PROPN
    NP -> PRON

# geändert (Spezifizierer-Regel = Phrasenabschluss):
    NP -> DET NOM
# geändert (rekursive PP-Adjunkt-Regel = nominale Zwischenebene):
    NOM -> NOM PP
# neu (Kopfregel = Phrasenkern):
    NOM -> N

    DET -> "die" | "ein"
    PROPN -> "Chomsky" | "Noam" | "Maria" | "Moritz"
    N -> "Studierende" | "Buch"
    PRON -> "ihnen"
    V -> "schenkte"
    P -> "von"
""")

In [15]:
sents = [
    "die Studierende schenkte ihnen ein Buch",
    "Maria schenkte ihnen ein Buch von Noam Chomsky"
]

parser = nltk.ChartParser(grammar)
for sent in sents:
    trees = list(parser.parse(sent.split()))
    if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
    else: print(f"no parse found for: {sent}")

                       S                       
     ┌─────────────────┴───────┐                
     │                         VP              
     │                 ┌───────┼────────┐       
     NP                │       │        NP     
 ┌───┴───────┐         │       │    ┌───┴───┐   
 │          NOM        │       NP   │      NOM 
 │           │         │       │    │       │   
DET          N         V      PRON DET      N  
 │           │         │       │    │       │   
die     Studierende schenkte ihnen ein     Buch

                      S                                
  ┌───────────────────┴───┐                             
  │                       VP                           
  │      ┌───────┬────────┴────┐                        
  │      │       │             NP                      
  │      │       │    ┌────────┴───┐                    
  │      │       │    │           NOM                  
  │      │       │    │   ┌────────┴────┐               
  │      │    

In [16]:
neg_sents = [
    "Studierende schenkte ihnen ein Buch",
    "die Maria schenkte ihnen ein Buch"
]

parser = nltk.ChartParser(grammar)
for sent in neg_sents:
    trees = list(parser.parse(sent.split()))
    if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
    else: print(f"no parse found for: {sent}")

no parse found for: Studierende schenkte ihnen ein Buch
no parse found for: die Maria schenkte ihnen ein Buch


### b) Ergänzen Sie die lexikalischen Regeln für einen entsprechenden Beispielsatz mit mehreren PP-Adjunkten. Erklären Sie die Ambiguität des Parsing-Ergebnisses.

#### Wählen Sie die PPs in Ihrem Beispielsatz so, dass die aus den syntaktischen Regeln folgende Ambiguität auch eine semantisch sinnvolle Interpretation hat.

In [17]:
sent = "Maria schenkte ihnen ein Buch aus dem Buchladen ohne einen Namen"

In [ ]:
# Erläuterung: 
# im Beispielsatz liegt PP-Attachment-Ambiguität im nominalen Bereich vor:
# das Buch bzw. der Buchladen hat keinen Namen

In [18]:
new_productions = [
    Production(Nonterminal('P'), ['aus']),
    Production(Nonterminal('DET'), ['dem']),
    Production(Nonterminal('N'), ['Buchladen']),
    Production(Nonterminal('P'), ['ohne']),
    Production(Nonterminal('DET'), ['einen']),
    Production(Nonterminal('N'), ['Namen']),
]

productions = copy.copy(grammar.productions())

for new_production in new_productions:
    productions.append(new_production)
    grammar_b = nltk.grammar.CFG(grammar.start(), productions)

print(grammar_b)

Grammar with 26 productions (start state = S)
    S -> NP VP
    VP -> V NP NP
    PP -> P NP
    NP -> PROPN
    NP -> PROPN PROPN
    NP -> PRON
    NP -> DET NOM
    NOM -> NOM PP
    NOM -> N
    DET -> 'die'
    DET -> 'ein'
    PROPN -> 'Chomsky'
    PROPN -> 'Noam'
    PROPN -> 'Maria'
    PROPN -> 'Moritz'
    N -> 'Studierende'
    N -> 'Buch'
    PRON -> 'ihnen'
    V -> 'schenkte'
    P -> 'von'
    P -> 'aus'
    DET -> 'dem'
    N -> 'Buchladen'
    P -> 'ohne'
    DET -> 'einen'
    N -> 'Namen'


In [19]:
parser = nltk.ChartParser(grammar_b)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

                 S                                                          
  ┌──────────────┴─────────────┐                                             
  │                            VP                                           
  │      ┌───────┬─────────────┴───────┐                                     
  │      │       │                     NP                                   
  │      │       │    ┌────────────────┴──────────┐                          
  │      │       │    │                          NOM                        
  │      │       │    │            ┌──────────────┴────────────┐             
  │      │       │    │           NOM                          │            
  │      │       │    │   ┌────────┴───┐                       │             
  │      │       │    │   │            PP                      PP           
  │      │       │    │   │    ┌───────┴───┐             ┌─────┴────┐        
  │      │       │    │   │    │           NP            │          NP

### c) Erweitern Sie die Grammatik abschließend um Regeln für rekursive PP-Adjunkte an die VP, um folgenden Beispielsatz parsen zu können. Halten Sie sich dabei an das X-Bar-Schema.

In [20]:
sent = "Maria schickte ihnen ein Buch aus München"

In [21]:
new_productions = [
    Production(Nonterminal('V'), ['schickte']),
    Production(Nonterminal('P'), ['aus']),
    Production(Nonterminal('PROPN'), ['München']),
    Production(Nonterminal('VP'), [Nonterminal('VERBAL')]), # Phrasenabschluss (hier ohne Spezifizierer)
    Production(Nonterminal('VERBAL'), [Nonterminal('VERBAL'), Nonterminal('PP')]), # rekursive PP-Adjunkt-Regel
    Production(Nonterminal('VERBAL'), [Nonterminal('VERBAL'), Nonterminal('NP'), Nonterminal('NP')]), # Komplement-Regel
    Production(Nonterminal('VERBAL'), [Nonterminal('V')]), # Kopfregel
]

productions = copy.copy(grammar_b.productions())

for new_production in new_productions:
    productions.append(new_production)
    grammar_c = nltk.grammar.CFG(grammar.start(), productions)

print(grammar_c)

Grammar with 33 productions (start state = S)
    S -> NP VP
    VP -> V NP NP
    PP -> P NP
    NP -> PROPN
    NP -> PROPN PROPN
    NP -> PRON
    NP -> DET NOM
    NOM -> NOM PP
    NOM -> N
    DET -> 'die'
    DET -> 'ein'
    PROPN -> 'Chomsky'
    PROPN -> 'Noam'
    PROPN -> 'Maria'
    PROPN -> 'Moritz'
    N -> 'Studierende'
    N -> 'Buch'
    PRON -> 'ihnen'
    V -> 'schenkte'
    P -> 'von'
    P -> 'aus'
    DET -> 'dem'
    N -> 'Buchladen'
    P -> 'ohne'
    DET -> 'einen'
    N -> 'Namen'
    V -> 'schickte'
    P -> 'aus'
    PROPN -> 'München'
    VP -> VERBAL
    VERBAL -> VERBAL PP
    VERBAL -> VERBAL NP NP
    VERBAL -> V


In [22]:
parser = nltk.ChartParser(grammar_c)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

         S                                           
  ┌──────┴──────────────┐                             
  │                     VP                           
  │                     │                             
  │                   VERBAL                         
  │              ┌──────┴──────────────────┐          
  │            VERBAL                      │         
  │      ┌───────┼────────────┐            │          
  │      │       │            NP           PP        
  │      │       │      ┌─────┴───┐    ┌───┴─────┐    
  NP   VERBAL    NP     │        NOM   │         NP  
  │      │       │      │         │    │         │    
PROPN    V      PRON   DET        N    P       PROPN 
  │      │       │      │         │    │         │    
Maria schickte ihnen   ein       Buch aus     München

         S                                          
  ┌──────┴─────────────┐                             
  │                    VP                           
  │                   